In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from nltk.corpus import reuters
from collections import defaultdict
from nltk import bigrams
from nltk.probability import FreqDist
from math import log

[nltk_data] Error loading stopwords: <urlopen error [Errno 54]
[nltk_data]     Connection reset by peer>
[nltk_data] Error loading punkt: <urlopen error [Errno 54] Connection
[nltk_data]     reset by peer>
[nltk_data] Error loading wordnet: <urlopen error [Errno 54]
[nltk_data]     Connection reset by peer>


In [3]:
train_df = pd.read_csv("/Users/vidurgoel/Downloads/archive-3/train.csv")
train_df.head()

,ind,activity_label,ctx_a,ctx_b,ctx,endings,source_id,split,split_type,label
0,4,Removing ice from car,"Then, the man writes over the snow covering th...",then,"Then, the man writes over the snow covering th...","[', the man adds wax to the windshield and cut...",activitynet~v_-1IBHYS3L-Y,train,indomain,3
1,8,Baking cookies,A female chef in white uniform shows a stack o...,the pans,A female chef in white uniform shows a stack o...,['contain egg yolks and baking soda.'\n 'are t...,activitynet~v_-2dxp-mv2zo,train,indomain,3
2,9,Baking cookies,A female chef in white uniform shows a stack o...,a knife,A female chef in white uniform shows a stack o...,['is seen moving on a board and cutting out it...,activitynet~v_-2dxp-mv2zo,train,indomain,3
3,12,Baking cookies,A tray of potatoes is loaded into the oven and...,a large tray of meat,A tray of potatoes is loaded into the oven and...,"['is placed onto a baked potato.'\n ', ls, and...",activitynet~v_-2dxp-mv2zo,train,indomain,3
4,27,Getting a haircut,The man in the center is demonstrating a hairs...,the man in the blue shirt,The man in the center is demonstrating a hairs...,['is standing on the sponge cutting the hair o...,activitynet~v_-JqLjPz-07E,train,indomain,2


In [4]:
train_df.isnull().sum()

ind                   0
activity_label        0
ctx_a                 0
ctx_b             25165
ctx                   0
endings               0
source_id             0
split                 0
split_type            0
label                 0
dtype: int64

In [5]:
train_df.drop(['ind', 'source_id', 'split', 'split_type'], axis=1, inplace=True)
train_df.head()

,activity_label,ctx_a,ctx_b,ctx,endings,label
0,Removing ice from car,"Then, the man writes over the snow covering th...",then,"Then, the man writes over the snow covering th...","[', the man adds wax to the windshield and cut...",3
1,Baking cookies,A female chef in white uniform shows a stack o...,the pans,A female chef in white uniform shows a stack o...,['contain egg yolks and baking soda.'\n 'are t...,3
2,Baking cookies,A female chef in white uniform shows a stack o...,a knife,A female chef in white uniform shows a stack o...,['is seen moving on a board and cutting out it...,3
3,Baking cookies,A tray of potatoes is loaded into the oven and...,a large tray of meat,A tray of potatoes is loaded into the oven and...,"['is placed onto a baked potato.'\n ', ls, and...",3
4,Getting a haircut,The man in the center is demonstrating a hairs...,the man in the blue shirt,The man in the center is demonstrating a hairs...,['is standing on the sponge cutting the hair o...,2


In [6]:
a0_list = []
a1_list = []
a2_list = []
a3_list = []
# split endings column into 4 separate columns iterating over row and checking for quotes
for i in range(len(train_df)):
    count = 0
    str_0 = ''
    str_1 = ''
    str_2 = ''
    str_3 = ''

    flag_single_quote = 0
    flag_double_quote = 0

    for j in train_df['endings'][i]:


        if(j == '\'' and flag_single_quote == 1):
            flag_single_quote = 0
            count += 1
            continue

        if(j == '\"' and flag_double_quote == 1):
            flag_double_quote = 0
            count += 1
            continue
        
        if(j == '\'' and flag_single_quote == 0 and flag_double_quote != 1):
            flag_single_quote = 1
            count += 1

        if(flag_single_quote == 1 and flag_double_quote != 1):
            if(count == 1):
                str_0 += j
            elif(count == 3):
                str_1 += j
            elif(count == 5):
                str_2 += j
            elif(count == 7):
                str_3 += j

        if(j == '\"' and flag_double_quote == 0 and flag_single_quote != 1):
            flag_double_quote = 1
            count += 1

        if(flag_double_quote == 1 and flag_single_quote != 1):
            if(count == 1):
                str_0 += j
            elif(count == 3):
                str_1 += j
            elif(count == 5):
                str_2 += j
            elif(count == 7):
                str_3 += j

    a0_list.append(str_0)
    a1_list.append(str_1)
    a2_list.append(str_2)
    a3_list.append(str_3)

In [7]:
train_df.drop(['endings'], axis=1, inplace=True)

# insert the 4 new columns into the dataframe before the labels column
train_df.insert(4, 'A0', a0_list)
train_df.insert(5, 'A1', a1_list)
train_df.insert(6, 'A2', a2_list)
train_df.insert(7, 'A3', a3_list)

train_df.head()

,activity_label,ctx_a,ctx_b,ctx,A0,A1,A2,A3,label
0,Removing ice from car,"Then, the man writes over the snow covering th...",then,"Then, the man writes over the snow covering th...","', the man adds wax to the windshield and cuts...","', a person board a ski lift, while two men su...","', the man puts on a christmas coat, knitted w...","', the man continues removing the snow on his ...",3
1,Baking cookies,A female chef in white uniform shows a stack o...,the pans,A female chef in white uniform shows a stack o...,'contain egg yolks and baking soda.,'are then sprinkled with brown sugar.,'are placed in a strainer on the counter.,'are filled with pastries and loaded into the ...,3
2,Baking cookies,A female chef in white uniform shows a stack o...,a knife,A female chef in white uniform shows a stack o...,'is seen moving on a board and cutting out its...,"'hits the peeled cheesecake, followed by slice...",'etches a shape into the inside of the baked p...,'is used to cut cylinder shaped dough into rou...,3
3,Baking cookies,A tray of potatoes is loaded into the oven and...,a large tray of meat,A tray of potatoes is loaded into the oven and...,'is placed onto a baked potato.,"', ls, and pickles are placed in the oven.",'is poured into a midden.,'is prepared then it is removed from the oven ...,3
4,Getting a haircut,The man in the center is demonstrating a hairs...,the man in the blue shirt,The man in the center is demonstrating a hairs...,'is standing on the sponge cutting the hair of...,'is doing the hairstyle with his hand and the ...,'sits on the chair next to the sink.,'is being shown eye to eye.,2


In [8]:
right_end=[]
for i in range(len(train_df)):
    if(train_df['label'][i]==0):
        right_end.append(a0_list[i])
    if(train_df['label'][i]==1):
        right_end.append(a1_list[i])
    if(train_df['label'][i]==2):
        right_end.append(a2_list[i])
    if(train_df['label'][i]==3):
        right_end.append(a3_list[i])   

In [9]:
modified_array = [s[1:] for s in right_end]

In [10]:
train_df.insert(8,'Ans',modified_array)
train_df.head()

,activity_label,ctx_a,ctx_b,ctx,A0,A1,A2,A3,Ans,label
0,Removing ice from car,"Then, the man writes over the snow covering th...",then,"Then, the man writes over the snow covering th...","', the man adds wax to the windshield and cuts...","', a person board a ski lift, while two men su...","', the man puts on a christmas coat, knitted w...","', the man continues removing the snow on his ...",", the man continues removing the snow on his car.",3
1,Baking cookies,A female chef in white uniform shows a stack o...,the pans,A female chef in white uniform shows a stack o...,'contain egg yolks and baking soda.,'are then sprinkled with brown sugar.,'are placed in a strainer on the counter.,'are filled with pastries and loaded into the ...,are filled with pastries and loaded into the o...,3
2,Baking cookies,A female chef in white uniform shows a stack o...,a knife,A female chef in white uniform shows a stack o...,'is seen moving on a board and cutting out its...,"'hits the peeled cheesecake, followed by slice...",'etches a shape into the inside of the baked p...,'is used to cut cylinder shaped dough into rou...,is used to cut cylinder shaped dough into rounds.,3
3,Baking cookies,A tray of potatoes is loaded into the oven and...,a large tray of meat,A tray of potatoes is loaded into the oven and...,'is placed onto a baked potato.,"', ls, and pickles are placed in the oven.",'is poured into a midden.,'is prepared then it is removed from the oven ...,is prepared then it is removed from the oven b...,3
4,Getting a haircut,The man in the center is demonstrating a hairs...,the man in the blue shirt,The man in the center is demonstrating a hairs...,'is standing on the sponge cutting the hair of...,'is doing the hairstyle with his hand and the ...,'sits on the chair next to the sink.,'is being shown eye to eye.,sits on the chair next to the sink.,2


In [11]:
train_df.drop(['ctx_a', 'ctx_b', 'A0', 'A1','A2','A3','label'], axis=1, inplace=True)
train_df.head()

,activity_label,ctx,Ans
0,Removing ice from car,"Then, the man writes over the snow covering th...",", the man continues removing the snow on his car."
1,Baking cookies,A female chef in white uniform shows a stack o...,are filled with pastries and loaded into the o...
2,Baking cookies,A female chef in white uniform shows a stack o...,is used to cut cylinder shaped dough into rounds.
3,Baking cookies,A tray of potatoes is loaded into the oven and...,is prepared then it is removed from the oven b...
4,Getting a haircut,The man in the center is demonstrating a hairs...,sits on the chair next to the sink.


In [12]:
copy_train_df=train_df

In [13]:
corpus=[]
for i in range(len(train_df)):
    corpus.append(train_df['ctx'][i]+' '+train_df['Ans'][i])
print(corpus[99])
print(train_df['ctx'][99])

A cheer-leading team begins to hold up posters as their mascot runs behind them. they then begin to do a routine and some of the girls run with streamers as the rest of them hold up the girls for their stunt.
A cheer-leading team begins to hold up posters as their mascot runs behind them. they


In [14]:
train_df.insert(3,'Sentence',corpus)
train_df.head()

,activity_label,ctx,Ans,Sentence
0,Removing ice from car,"Then, the man writes over the snow covering th...",", the man continues removing the snow on his car.","Then, the man writes over the snow covering th..."
1,Baking cookies,A female chef in white uniform shows a stack o...,are filled with pastries and loaded into the o...,A female chef in white uniform shows a stack o...
2,Baking cookies,A female chef in white uniform shows a stack o...,is used to cut cylinder shaped dough into rounds.,A female chef in white uniform shows a stack o...
3,Baking cookies,A tray of potatoes is loaded into the oven and...,is prepared then it is removed from the oven b...,A tray of potatoes is loaded into the oven and...
4,Getting a haircut,The man in the center is demonstrating a hairs...,sits on the chair next to the sink.,The man in the center is demonstrating a hairs...


In [15]:
train_df.drop(['ctx', 'Ans'], axis=1, inplace=True)
train_df.head()

,activity_label,Sentence
0,Removing ice from car,"Then, the man writes over the snow covering th..."
1,Baking cookies,A female chef in white uniform shows a stack o...
2,Baking cookies,A female chef in white uniform shows a stack o...
3,Baking cookies,A tray of potatoes is loaded into the oven and...
4,Getting a haircut,The man in the center is demonstrating a hairs...


In [16]:
num_unique_labels = train_df['activity_label'].nunique()
print(num_unique_labels)
print(len(train_df))

178
39905


In [49]:
# dictionary_containing_sentence_of_each_activity_label
train_df.head()
dictionary_containing_sentence_of_each_activity_label={}
for i in range(len(train_df)):
    if train_df.loc[i,'activity_label'] in dictionary_containing_sentence_of_each_activity_label:
        dictionary_containing_sentence_of_each_activity_label[train_df.loc[i,'activity_label']].append(train_df.loc[i,'Sentence'])
    else:
        dictionary_containing_sentence_of_each_activity_label[train_df.loc[i,'activity_label']]=[]
        dictionary_containing_sentence_of_each_activity_label[train_df.loc[i,'activity_label']].append(train_df.loc[i,'Sentence'])
print(len(dictionary_containing_sentence_of_each_activity_label))
print(dictionary_containing_sentence_of_each_activity_label['Baking cookies'])
train_df.head()
# print(dictionary_containing_sentence_of_each_activity_label)

178
['A female chef in white uniform shows a stack of baking pans in a large kitchen presenting them. the pans are filled with pastries and loaded into the oven.', 'A female chef in white uniform shows a stack of baking pans in a large kitchen presenting them. The pans are filled with pastries and loaded into the oven. a knife is used to cut cylinder shaped dough into rounds.', 'A tray of potatoes is loaded into the oven and removed. A large tray of cake is flipped over and placed on counter. a large tray of meat is prepared then it is removed from the oven by a helper when done.', 'The woman is talking again and the next cookie she is shown making is called "cookie snowballs" and it shows all the ingredients needed, a list on how long to cook it for, and she\'s demonstrating step by step until it\'s all baked and on a plate next to the other cookies. the video ends with more pictures of other cookies she wasn\'t shown making, then to a green screen that has white words that include a 

,activity_label,Sentence
0,Removing ice from car,"Then, the man writes over the snow covering th..."
1,Baking cookies,A female chef in white uniform shows a stack o...
2,Baking cookies,A female chef in white uniform shows a stack o...
3,Baking cookies,A tray of potatoes is loaded into the oven and...
4,Getting a haircut,The man in the center is demonstrating a hairs...


In [50]:
X = train_df['Sentence']

m=""

for i in X:
    m=m+" "+i
print(m[0:1000])
corpus=m

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
set1={'[header]', '[title]', '[step]', '[substep]', 'header', 'title', 'step', 'substep'}
tokens = nltk.word_tokenize(corpus.lower())
filtered_documents = []
for word in tokens:
    if word=='.':
        filtered_documents.append('<e>')
        # filtered_documents.append('<s>')
        continue
    if word.isalnum() and word not in set1:
        filtered_documents.append(word)
# filtered_documents = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in set1]
tokens=filtered_documents

# Create unigrams, bigrams, and trigrams
unigrams = list(ngrams(tokens, 1))
bigrams = list(ngrams(tokens, 2))
trigrams = list(ngrams(tokens, 3))
quadgrams = list(ngrams(tokens, 4))
pentagrams = list(ngrams(tokens, 5))
hexagrams = list(ngrams(tokens, 6))

# Calculate frequencies for each n-gram with Laplace smoothing
V = len(set(tokens))  # Vocabulary size
unigram_freq = nltk.FreqDist(unigrams)
bigram_freq = nltk.FreqDist(bigrams)
trigram_freq = nltk.FreqDist(trigrams)
quadgrams_freq = nltk.FreqDist(quadgrams)
pentagrams_freq = nltk.FreqDist(pentagrams)
hexagrams_freq = nltk.FreqDist(hexagrams)

n_gram_array=[]
n_gram_array.append(unigram_freq)
n_gram_array.append(bigram_freq)
n_gram_array.append(trigram_freq)
n_gram_array.append(quadgrams_freq)
n_gram_array.append(pentagrams_freq)
n_gram_array.append(hexagrams_freq)


def laplace_smoothed_prob(ngram, ngram_freq, n):
    # Apply Laplace smoothing to calculate probability
    numerator = ngram_freq[ngram] + 1
    denominator = ngram_freq[ngram[:-1]] + V
    return n * numerator / denominator

# Function to predict the next word(s) with Laplace smoothing
def predict_next_word(input_text, ngram_model, n):
    tokens = nltk.word_tokenize(input_text.lower())
    filtered_documents = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in set1]
    input_tokens=filtered_documents
    
    # Replace out-of-vocabulary words with low probability words from the corpus
    replaced_tokens = []
    for i in range(len(input_tokens)):
        if input_tokens[i] not in set(tokens):
            low_prob_words = sorted(set(tokens), key=lambda x: laplace_smoothed_prob((x,), unigram_freq, 1))
            replaced_tokens.append(low_prob_words[0] if low_prob_words else None)
        else:
            replaced_tokens.append(input_tokens[i])
    
    # Extend the training corpus with the replaced tokens
    tokens.extend(replaced_tokens)
    
    # Filter n-grams that start with the updated input_ngram
    input_ngram = tuple(input_tokens[-n + 1:])
    candidate_ngrams = [ngram for ngram in ngram_model if ngram[:n - 1] == input_ngram]
    
    if not candidate_ngrams:
        return None
    
    # Select the most likely next word(s) based on smoothed probability
    next_ngram = max(candidate_ngrams, key=lambda x: laplace_smoothed_prob(x, ngram_model, n))
    
    return next_ngram[n - 1]

 Then, the man writes over the snow covering the window of a car, and a woman wearing winter clothes smiles. then , the man continues removing the snow on his car. A female chef in white uniform shows a stack of baking pans in a large kitchen presenting them. the pans are filled with pastries and loaded into the oven. A female chef in white uniform shows a stack of baking pans in a large kitchen presenting them. The pans are filled with pastries and loaded into the oven. a knife is used to cut cylinder shaped dough into rounds. A tray of potatoes is loaded into the oven and removed. A large tray of cake is flipped over and placed on counter. a large tray of meat is prepared then it is removed from the oven by a helper when done. The man in the center is demonstrating a hairstyle on the person wearing the blue shirt. the man in the blue shirt sits on the chair next to the sink. The man in the blue shirt sits on the chair next to the sink. The other man begins washing his hair. he scrubs

Predicting the end of the sentence

In [51]:
def wordgen(words):
    empty=[]
    for i in range(len(words[-1])):
        input_text=words[-1][i]
        for i in range(1,6):
            text=predict_next_word(input_text, n_gram_array[i], i)
            if(text != None):
                empty.append(input_text +' '+text)
    set1=set(empty)
    list1=list(set1)
    words.append(list1)
    return words

In [82]:
import random
def sentences_generator(input_text):
    words=[]
    empty=[]

    for i in range(1,6):
        text=predict_next_word(input_text, n_gram_array[i], i)
        if(text != None):
            # print(text)
            empty.append(input_text+' '+text)
    additional_words_list=["the", "a", "with", "and", "which", "that", "is", "was", "will", "should"]
    if(len(empty)==0):
        selected_list = random.sample(additional_words_list, 4)
        for i in selected_list:
            empty.append(input_text+' '+i)
    words.append(empty)
    sentence_list = []
    sentence_ended = []
    for i in range(5):
        # print(words)
        words=wordgen(words)
        # print(words)
        # print("Length of words", len(words))
        # print(len(words[0]))
        for j in words:
            for k in j:
                # sentence_list.append(k)
                if k not in sentence_list:
                    sentence_list.append(k)
                
                # if there is '<e>' in k then remove k from words
                if '<e>' in k:
                    sentence_ended.append(k)
                    # j.remove(k)
    a7=sentence_list+sentence_ended
    return a7



input_text="Dark matter gathers around"
sentence_list=sentences_generator(input_text)
print(sentence_list)


['Dark matter gathers around <e>', 'Dark matter gathers around as', 'Dark matter gathers around as group', 'Dark matter gathers around as man', 'Dark matter gathers around <e> formado', 'Dark matter gathers around as pot', 'Dark matter gathers around as pot to', 'Dark matter gathers around as pot of', 'Dark matter gathers around <e> formado sozo', 'Dark matter gathers around as man standing', 'Dark matter gathers around as group of', 'Dark matter gathers around as man is', 'Dark matter gathers around as pot of people', 'Dark matter gathers around as pot of water', 'Dark matter gathers around as pot to see', 'Dark matter gathers around as pot to boil', 'Dark matter gathers around as man is seen', 'Dark matter gathers around as man standing in', 'Dark matter gathers around as group of people', 'Dark matter gathers around <e> formado sozo appears', 'Dark matter gathers around as pot to the', 'Dark matter gathers around as man is standing', 'Dark matter gathers around as pot to the camera'

Ranking and printing the top 15 predictions

In [83]:
# Calculate average PMI for each sentence
import nltk
from nltk import bigrams
from nltk.probability import FreqDist
from math import log


def printing(corpus,sentence_list):
    tokens = nltk.word_tokenize(corpus.lower())
    filtered_documents = [word for word in tokens if word.isalnum()]
    corpus_bigram_list = list(bigrams(filtered_documents))

    words_freq_distr = FreqDist(filtered_documents)
    bigrams_freq_distr = FreqDist(corpus_bigram_list)
    pmi_scores_of_all_sentences_a1 = []
    for s in sentence_list:
        s_tokens = nltk.word_tokenize(s.lower())
        s_bigrams = list(bigrams(s_tokens))
        total_pmi_score=0
        for w1,w2 in s_bigrams:
            
            tot=len(filtered_documents)
            prob_x = words_freq_distr[w1] / tot
            prob_y = words_freq_distr[w2]  / tot

            if (w1, w2) in bigrams_freq_distr:
                prob_x_and_y = bigram_freq[(w1, w2)] / len(corpus_bigram_list)
            else:
                prob_x_and_y = 0.0

            if prob_x_and_y != 0.0:
                total_pmi_score+=log(prob_x_and_y/ (prob_x * prob_y), 2)

        if len(s_bigrams) > 0:
            pmi_mean = total_pmi_score / len(s_bigrams)
        else:
            pmi_mean = 0.0
        pmi_scores_of_all_sentences_a1.append(pmi_mean)
    # a1=sentence_list+sentence_ended
    # print(type(a1))
    # pmi_scores_of_all_sentence_a1 = printing(corpus,a1)
    sorted_pmi_scores_sentence_a1 = sorted(zip(sentence_list, pmi_scores_of_all_sentences_a1), key=lambda x: x[1], reverse=True)

    main_list1={}
    main_list2={}
    for rank, (s, pmi) in enumerate(sorted_pmi_scores_sentence_a1, start=1):
        if s.endswith(" <e>"):
            main_list1[s]=pmi
        else:
            main_list2[s]=pmi
    print("Printing the top ranked sentences according to the Mean PMI scores(sentences ended with <e>):")
    num=1
    for i in main_list1:
        print(f"Rank number {num}: {i} (Mean PMI score: {main_list1[i]})")
        num+=1
        # if(num>10):
        #     break
    num=1
    print()
    print()
    print("Printing the top ranked sentences according to the Mean PMI scores(sentences not ended with <e>):")
    for i in main_list2:
        print(f"Rank number {num}: {i} (Mean PMI score: {main_list2[i]})")
        num+=1
        # if(num>10):
        #     break
        
printing(corpus,sentence_list)


Printing the top ranked sentences according to the Mean PMI scores(sentences ended with <e>):
Rank number 1: Dark matter gathers around as pot of water <e> (Mean PMI score: 1.0105366317728843)
Rank number 2: Dark matter gathers around as pot to boil water <e> (Mean PMI score: 0.9772780159187804)
Rank number 3: Dark matter gathers around <e> (Mean PMI score: 0.8345536507440986)
Rank number 4: Dark matter gathers around as pot to the camera <e> (Mean PMI score: 0.7982805382562234)
Rank number 5: Dark matter gathers around as pot to boil <e> (Mean PMI score: 0.6687697614959551)


Printing the top ranked sentences according to the Mean PMI scores(sentences not ended with <e>):
Rank number 1: Dark matter gathers around <e> formado sozo appears (Mean PMI score: 4.268440239223049)
Rank number 2: Dark matter gathers around <e> formado sozo appears on (Mean PMI score: 4.257055850513522)
Rank number 3: Dark matter gathers around <e> formado sozo appears on the (Mean PMI score: 4.111744992970708)

Testing Script

In [81]:
file_name='/Users/vidurgoel/Desktop/Ml_project/a1.txt'
with open(file_name, 'r') as file:
    lines = [line.strip() for line in file]
print(lines)
for line in lines:
    print("Given line: ",line)
    print()
    sentence_list=sentences_generator(line)
    printing(corpus,sentence_list)
    print()
    print()
    print()

['A female chef in white uniform shows a stack of baking pans in a large kitchen presenting them. the pans', 'The man in the center is demonstrating a hairstyle on the person wearing the blue shirt. the man in the blue shirt', 'The man in the blue shirt sits on the chair next to the sink. The other man begins washing his hair. he', 'Two bodybuilder women are seated at a table. they', 'Two bodybuilder women are seated at a table. They are arm wrestling, vieing to win. when there', 'This is a tutorial on how to start a campfire. it', 'This is a tutorial on how to start a campfire. It shows the campfire burning on the ground. it', 'It shows the campfire burning on the ground. It starts with the things you will need to begin with. a designated place and some rocks', '.a lady named linda, creator of paint along', 'A lady named linda, creator of paint along is demonstrating how to do an acrylic painting. she']
Given line:  A female chef in white uniform shows a stack of baking pans in a larg